# This example demonstrate how to train region detector model (standart image classification model).

For train datasets please download last version of dataset [https://nomeroff.net.ua/datasets/](https://nomeroff.net.ua/datasets/). Unpack archive and rename to **./datasets/** .
For examle
```bash
cd ./datasets
wget https://nomeroff.net.ua/datasets/autoriaNumberplateDataset-2021-03-01.zip
unzip autoriaNumberplateDataset-2021-03-01.zip
mv autoriaNumberplateDataset-2021-03-01 np
```
or use your own dataset.

After that you have to convert the dataset to yolo format:

**nomeroff-net/train/datasetConvertors/from_MaskRCNN_datafromat_to_YOLO_dataformat.ipynb**

In [ ]:
!cd yolov5; python3 train.py --data np.yaml --cfg yolov5s.yaml --batch-size 64;